In [ ]:
import os, re
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Bidirectional, GRU
from keras.layers import GlobalMaxPool1D, Conv1D, MaxPooling1D, Flatten, AveragePooling1D
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import AUC
from keras.models import load_model
import os, re
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import nltk
from sklearn.utils import shuffle

from string import digits, punctuation

from keras.preprocessing.text import Tokenizer 
from nltk.tokenize.casual import reduce_lengthening
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from keras.preprocessing.sequence import pad_sequences

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/drive/My Drive/TESI/dataset.csv')

In [ ]:
len(data)

124207

In [ ]:
data['TESTO'].isnull().any()

False

In [ ]:
isacco = pd.read_csv('/content/drive/My Drive/TESI/ISACCO.csv')

In [ ]:
isacco['TESTO'].isnull().any()

False

In [ ]:
isacco.head()

,Unnamed: 0,TESTO,DOPPIE,INV. OMIS. INS. LETT/SILL,FUS./SEP ILL.,SCAMBIO OMOF.,H,ACCENTI
0,0,La statua nel parco C'era una volta una statu...,0,0,0,0,0,0
1,1,La statua nel parco C'era una volta una statu...,0,0,0,0,0,0
2,2,La statua nel parco In un parco c'era una sta...,0,0,0,0,0,0
3,3,La statua nel parco C'era una volta una statu...,0,0,0,0,0,0
4,4,La statua nel parco C'era una volta in un par...,0,0,0,0,0,0


In [ ]:
no_dys = pd.read_csv('/content/drive/My Drive/TESI/no_dysorthography.csv', sep=';')

In [ ]:
no_dys

,TESTO,DISORTOGRAFICO
0,L’autunno comincia con l’equinozio che cade in...,NaN
1,La rana salta in alto. Paolo legge il giornale...,NaN
2,Un’ape si è posata sul fiore. Ai bambini piace...,NaN
3,La leggenda di Isabella coccinella e Gastone C...,NaN
4,Mi chiamo Francesca e lavoro come segretaria n...,NaN
5,Una mattina speciale. Questa mattina sulle pis...,NaN
6,La giornata degli animali. Michele è un bambin...,NaN
7,"Sui tetti della mia città, c’è una gatta che n...",NaN
8,"La cuoca Pasqualina non sa cucinare, combina t...",NaN
9,"Cara Eugenia, gli ippopotami non mangiano carn...",NaN


In [ ]:
no_dys.isnull().any()

TESTO             False
DISORTOGRAFICO     True
dtype: bool

In [ ]:
data = data.append(isacco)

In [ ]:
data

,Unnamed: 0,TESTO,DOPPIE,INV. OMIS. INS. LETT/SILL,FUS./SEP ILL.,SCAMBIO OMOF.,H,ACCENTI,DISORTOGRAFICO
0,0,Filippo un ragazzo che abita in un piccolo qua...,1,2,3,1,0,2,1.0
1,1,I bisonti rappresentavano la principale fonte ...,0,2,3,3,0,0,1.0
2,2,Se al calar della notte si avvicinavano ad un ...,0,2,3,3,0,0,1.0
3,3,I bisonti rappresentavano la principale fonte ...,0,2,3,3,0,0,1.0
4,4,Quando i bisonti cominciavano a correre (il ru...,0,2,3,3,0,0,1.0
...,...,...,...,...,...,...,...,...,...
119,119,C'era una volta in un parco una statua di pie...,0,60,0,0,0,0,NaN
120,120,La storia nel parco C'era una volta un geogra...,0,15,0,0,0,2,NaN
121,121,Al centro di un parco c'era una statua di pie...,3,34,0,0,0,0,NaN
122,122,La statua nel prato C'era una volta una statu...,2,6,0,0,0,0,NaN


In [ ]:
#x_train = data['comment_text']
x_train = data['TESTO']

x_train

0      Filippo un ragazzo che abita in un piccolo qua...
1      I bisonti rappresentavano la principale fonte ...
2      Se al calar della notte si avvicinavano ad un ...
3      I bisonti rappresentavano la principale fonte ...
4      Quando i bisonti cominciavano a correre (il ru...
                             ...                        
119     C'era una volta in un parco una statua di pie...
120     La storia nel parco C'era una volta un geogra...
121     Al centro di un parco c'era una statua di pie...
122     La statua nel prato C'era una volta una statu...
123     Un giorno dei ragazzi  butarono  giu  un mapp...
Name: TESTO, Length: 124331, dtype: object

In [ ]:
data.loc[data["DOPPIE"]+data["INV. OMIS. INS. LETT/SILL"]+data["FUS./SEP ILL."]+data["SCAMBIO OMOF."]+data["H"]+data["ACCENTI"]<=2, "SCORE"] = 0
data.loc[data["DOPPIE"]+data["INV. OMIS. INS. LETT/SILL"]+data["FUS./SEP ILL."]+data["SCAMBIO OMOF."]+data["H"]+data["ACCENTI"]<=5, "SCORE"] = 1
data.loc[data["DOPPIE"]+data["INV. OMIS. INS. LETT/SILL"]+data["FUS./SEP ILL."]+data["SCAMBIO OMOF."]+data["H"]+data["ACCENTI"]<=8, "SCORE"] = 2
data.loc[data["DOPPIE"]+data["INV. OMIS. INS. LETT/SILL"]+data["FUS./SEP ILL."]+data["SCAMBIO OMOF."]+data["H"]+data["ACCENTI"]>=9, "SCORE"] = 3

In [ ]:
data

,Unnamed: 0,TESTO,DOPPIE,INV. OMIS. INS. LETT/SILL,FUS./SEP ILL.,SCAMBIO OMOF.,H,ACCENTI,DISORTOGRAFICO,SCORE
0,0,Filippo un ragazzo che abita in un piccolo qua...,1,2,3,1,0,2,1.0,3.0
1,1,I bisonti rappresentavano la principale fonte ...,0,2,3,3,0,0,1.0,2.0
2,2,Se al calar della notte si avvicinavano ad un ...,0,2,3,3,0,0,1.0,2.0
3,3,I bisonti rappresentavano la principale fonte ...,0,2,3,3,0,0,1.0,2.0
4,4,Quando i bisonti cominciavano a correre (il ru...,0,2,3,3,0,0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...
119,119,C'era una volta in un parco una statua di pie...,0,60,0,0,0,0,NaN,3.0
120,120,La storia nel parco C'era una volta un geogra...,0,15,0,0,0,2,NaN,3.0
121,121,Al centro di un parco c'era una statua di pie...,3,34,0,0,0,0,NaN,3.0
122,122,La statua nel prato C'era una volta una statu...,2,6,0,0,0,0,NaN,2.0


In [ ]:
col = ['DOPPIE', 'INV. OMIS. INS. LETT/SILL', 'FUS./SEP ILL.', 'SCAMBIO OMOF.', 'H', 'ACCENTI']
#col = ['subj',	'opos',	'oneg'	,'iro',	'lpos',	'lneg',	'top'	]
y_train = data['SCORE']

In [ ]:
# control of null elements

x_train.isnull().any()

False

In [ ]:
y_train.isnull().any()

False

In [ ]:
y_train = y_train.values

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
max_features = 30000

tokenizer = Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(list(x_train))

In [ ]:
np.mean(x_train.str.len()), np.std(x_train.str.len())

(683.6265613563794, 298.79798926672174)

In [ ]:
x_tr_tokenized = tokenizer.texts_to_sequences(x_train)

In [ ]:

max_len = 600
X_tr_tok = pad_sequences(x_tr_tokenized, maxlen=max_len)

In [ ]:
def build_model():
    """
    Questa funzione crea il modello e lo compila
    return: a model
    
    emb_matrix: matrice che inizializza i pesi dell'Embedding layer
    
    """
    
    inp = Input(shape=(max_len,))
    emb = Embedding(max_features, 200, input_length=max_len)(inp)
    d1 = Dense(units=100, activation='relu')(emb)
    x = Flatten()(d1)
    out = Dense(4, activation='sigmoid')(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC()])
    
    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 600)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 600, 200)          6000000   
_________________________________________________________________
dense (Dense)                (None, 600, 100)          20100     
_________________________________________________________________
flatten (Flatten)            (None, 60000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 240004    
Total params: 6,260,104
Trainable params: 6,260,104
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cp = ModelCheckpoint('/content/drive/My Drive/TESI/model_SCORE.h5',verbose=1, monitor='val_auc', mode='max')

In [ ]:
history1_1 = model.fit(X_tr_tok, y_train, batch_size=128, validation_split=0.1,epochs=2, verbose=1, callbacks=[cp])

Epoch 1/2
875/875 [==============================] - ETA: 0s - loss: 0.0194 - auc: 0.9997
Epoch 00001: saving model to /content/drive/My Drive/TESI/model_SCORE.h5
875/875 [==============================] - 112s 128ms/step - loss: 0.0194 - auc: 0.9997 - val_loss: 0.7892 - val_auc: 0.9039
Epoch 2/2
875/875 [==============================] - ETA: 0s - loss: 1.7991e-05 - auc: 1.0000
Epoch 00002: saving model to /content/drive/My Drive/TESI/model_SCORE.h5
875/875 [==============================] - 114s 130ms/step - loss: 1.7991e-05 - auc: 1.0000 - val_loss: 0.9148 - val_auc: 0.8605


In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('/content/drive/My Drive/TESI/model_SCORE.h5')

In [ ]:
import pandas as pd

In [ ]:
test_set = pd.read_csv('/content/drive/My Drive/TESI/test_set.csv', sep=';')

In [ ]:
x = ['']

In [ ]:
col = ['SCORE1', 'SCORE2', 'SCORE3', 'SCORE4']
y = test_set[col]

In [ ]:
max_features = 30000

tokenizer = Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(list(x))

In [ ]:
x_tr_tokenized = tokenizer.texts_to_sequences(x)

In [ ]:
max_len = 600
X_tr_tok = pad_sequences(x_tr_tokenized, maxlen=max_len)

In [ ]:
np.argmax(model.predict(X_tr_tok))

2